# Human-in-the-loop（HITL）是什麼？

Human-in-the-loop 的作用是讓人類參與決策，補充或修正系統狀態：
> 每個 HITL 接收 state → 向使用者詢問或請使用者確認 → 回傳要更新的 state 部分欄位。


# Interrupt

LangGraph 為 HITL 設計了一個節點 Interrupt。作用是在流程中途暫停或改變執行，根據條件觸發特定動作。
> interrupt：讓 Graph 停下來，直到人類傳入新的 state 再繼續往下跑

## 基本用法
### 1. 定義 State 和 Node


In [9]:
from typing import TypedDict, Optional
from langgraph.types import interrupt

class AgentState(TypedDict):
    approved: Optional[bool]

def approval_node(state: AgentState):
    """
    範例 Node：請使用者確認操作是否批准
    """
    # HITL：暫停並向使用者詢問
    approved = interrupt("Do you approve this action? (yes/no)")
    return {"approved": approved}

說明：

- interrupt() 會暫停 graph 執行，等待外部輸入
- 參數是提示訊息，會顯示給使用者
- 返回值是使用者的輸入

### 2. 建立和執行 Graph
執行時，需要 MemorySaver（後面會有章節詳細介紹 Memory 管理）：

In [16]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

# 建立 graph
workflow = StateGraph(AgentState)
workflow.add_node("approval", approval_node)
workflow.set_entry_point("approval")
workflow.set_finish_point("approval")

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}

result = graph.invoke({"approved": None}, config)

user_input = input("Do you approve this action? (yes/no): ")
approved = user_input.lower() == "yes" # 將使用者輸入轉換為布林值

result = graph.invoke(
    Command(resume=approved),  # 將使用者的回應傳入
    config
)
print("Final Result:", result)

Final Result: {'approved': True}


說明：
- config：包含 thread_id，用來識別對話並存取狀態
- checkpointer：儲存 graph 狀態的工具，interrupt 必須搭配使用
- Command(resume=...)：在 graph 因 interrupt 暫停後，用來將外部輸入傳回並從中斷點繼續執行


流程：

1. 第一次執行：graph 跑到 interrupt() 時會拋出異常來停止執行
2. 儲存狀態：checkpointer 把當前狀態存起來
3. 等待輸入：你的程式可以拿到使用者輸入
4. 第二次執行：用同個 thread_id 再次呼叫，從暫停處繼續


## 根據結果跳轉到不同 Node

# to be continued